In [ ]:
!pip install -q peft transformers datasets

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [12]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]

  0%|          | 0/2 [00:00<?, ?it/s]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [13]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)

3


In [15]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

In [17]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [18]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [22]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 425/425 [00:54<00:00,  7.84it/s]


epoch=0: train_ppl=tensor(1046.5624, device='cuda:0') train_epoch_loss=tensor(6.9533, device='cuda:0') eval_ppl=tensor(3409.0623, device='cuda:0') eval_epoch_loss=tensor(8.1342, device='cuda:0')


100%|██████████| 425/425 [00:54<00:00,  7.83it/s]


epoch=1: train_ppl=tensor(255.4648, device='cuda:0') train_epoch_loss=tensor(5.5431, device='cuda:0') eval_ppl=tensor(5251.0288, device='cuda:0') eval_epoch_loss=tensor(8.5662, device='cuda:0')


100%|██████████| 425/425 [00:54<00:00,  7.82it/s]


...

epoch=47: train_ppl=tensor(1.1126, device='cuda:0') train_epoch_loss=tensor(0.1067, device='cuda:0') eval_ppl=tensor(73729.0234, device='cuda:0') eval_epoch_loss=tensor(11.2082, device='cuda:0')


100%|██████████| 425/425 [00:54<00:00,  7.84it/s]


epoch=48: train_ppl=tensor(1.1222, device='cuda:0') train_epoch_loss=tensor(0.1153, device='cuda:0') eval_ppl=tensor(75047.0234, device='cuda:0') eval_epoch_loss=tensor(11.2259, device='cuda:0')


100%|██████████| 425/425 [00:54<00:00,  7.83it/s]

epoch=49: train_ppl=tensor(1.1150, device='cuda:0') train_epoch_loss=tensor(0.1089, device='cuda:0') eval_ppl=tensor(75047.0234, device='cuda:0') eval_epoch_loss=tensor(11.2259, device='cuda:0')


In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
peft_model_id = "likhith231/bloomz-560m_PROMPT_TUNING_CAUSAL_LM_50"
model.push_to_hub("likhith231/bloomz-560m_PROMPT_TUNING_CAUSAL_LM_50", use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/likhith231/bloomz-560m_PROMPT_TUNING_CAUSAL_LM_50/commit/296795ece4345723b7f06531034b68d094b3a8fa', commit_message='Upload model', commit_description='', oid='296795ece4345723b7f06531034b68d094b3a8fa', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
i=10
f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : '


'Tweet text : @BTCare Yes but was cut off. Can someone come out? Label : '

In [31]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


model = AutoPeftModelForCausalLM.from_pretrained("likhith231/bloomz-560m_PROMPT_TUNING_CAUSAL_LM_50").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-560m")

inputs = tokenizer(
    "Tweet text : Couples wallpaper, so cute. :) #BrothersAtHome. Label : ",
    return_tensors="pt",
)

In [32]:
import torch
with torch.no_grad():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : Couples wallpaper, so cute. :) #BrothersAtHome. Label : no complaint']
